# Byte Pair Encoding (BPE) Tokenization Tutorial

Welcome to this comprehensive tutorial on Byte Pair Encoding (BPE), one of the most important tokenization algorithms in modern Natural Language Processing!

## What You'll Learn

1. What tokenization is and why it matters
2. The problem BPE solves
3. How the BPE algorithm works step-by-step
4. Implementing BPE from scratch in Python
5. How BPE is used in modern language models

---

## 1. Introduction to Tokenization

**Tokenization** is the process of breaking text into smaller units called "tokens". These tokens are the fundamental units that language models process.

### Why Do We Need Tokenization?

Language models work with numbers, not text. Tokenization converts text into numerical representations that models can understand.

### Common Tokenization Approaches:

1. **Character-level**: Each character is a token
   - Pros: Small vocabulary, handles any text
   - Cons: Very long sequences, loses word meaning

2. **Word-level**: Each word is a token
   - Pros: Preserves word meaning
   - Cons: Huge vocabulary, can't handle unknown words

3. **Subword-level**: Words are broken into meaningful parts (BPE falls here!)
   - Pros: Balance between vocabulary size and sequence length
   - Cons: More complex implementation

**BPE is used in GPT, BERT, RoBERTa, and many other modern language models!**

## 2. The Problem BPE Solves

Let's see the problems with simple approaches:

In [1]:
# Example text
text = "The quick brown fox jumps over the lazy dog. The fox is quick!"

# Character-level tokenization
char_tokens = list(text)
print(f"Character tokens ({len(char_tokens)} tokens):")
print(char_tokens[:30], "...\n")

# Word-level tokenization
word_tokens = text.lower().split()
print(f"Word tokens ({len(word_tokens)} tokens):")
print(word_tokens)
print(f"\nUnique characters: {len(set(char_tokens))}")
print(f"Unique words: {len(set(word_tokens))}")

Character tokens (62 tokens):
['T', 'h', 'e', ' ', 'q', 'u', 'i', 'c', 'k', ' ', 'b', 'r', 'o', 'w', 'n', ' ', 'f', 'o', 'x', ' ', 'j', 'u', 'm', 'p', 's', ' ', 'o', 'v', 'e', 'r'] ...

Word tokens (13 tokens):
['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog.', 'the', 'fox', 'is', 'quick!']

Unique characters: 30
Unique words: 10


### Problems:

- **Character-level**: 58 tokens for a short sentence! Sequences get very long.
- **Word-level**: What about "jumping" vs "jumps"? They're treated as completely different words, even though they share meaning.
- **Unknown words**: Word-level can't handle words not in the vocabulary.

**BPE Solution**: Learn frequent subword units automatically from the data!

## 3. How BPE Works: The Algorithm

BPE is beautifully simple! It works in two phases:

### Training Phase:

1. Start with a vocabulary of all individual characters
2. Split all words into sequences of characters
3. Repeatedly:
   - Find the most frequent pair of adjacent tokens
   - Merge this pair into a new token
   - Add the new token to the vocabulary
4. Stop after a fixed number of merges (hyperparameter)

### Encoding Phase:

1. Start with word split into characters
2. Apply learned merge rules in order
3. Result: word encoded as sequence of subword tokens

### Visual Example:

```
Corpus: "low" (5 times), "lower" (2 times), "newest" (6 times), "widest" (3 times)

Initial tokens (characters): l o w e r n w i d s t

Step 1: Most frequent pair is 'e' + 's' → merge into 'es'
  "newest" → n e w e s t → n e w es t
  "widest" → w i d e s t → w i d es t

Step 2: Most frequent pair is 'es' + 't' → merge into 'est'
  "newest" → n e w est
  "widest" → w i d est

Step 3: Most frequent pair is 'l' + 'o' → merge into 'lo'
  "low" → lo w
  "lower" → lo w e r

... and so on!
```

## 4. Implementing BPE from Scratch

Let's build our own BPE tokenizer step by step!

In [2]:
from collections import defaultdict, Counter
from heapq import merge
import re

class BytePairEncoding:
    def __init__(self, num_merges=10):
        """
        Initialize BPE tokenizer.
        
        Args:
            num_merges: Number of merge operations to perform
        """
        self.num_merges = num_merges
        self.merges = {}  # Store merge rules: (pair) -> merged_token
        self.vocab = {}   # Final vocabulary
        
    def get_pairs(self, word):
        """
        Get all adjacent pairs of tokens in a word.
        
        Args:
            word: List of tokens
            
        Returns:
            Set of adjacent pairs
        """
        pairs = set()
        prev_token = word[0]
        for token in word[1:]:
            pairs.add((prev_token, token))
            prev_token = token
        # print("PAIRS: ",pairs)
        return pairs
    
    def train(self, corpus):
        """
        Train BPE on a corpus of text.
        
        Args:
            corpus: List of words or string of text
        """
        # Starts with character-level tokens

        # Iteratively finds most frequent pairs across the entire corpus

        # Creates and stores merge rules: ('w', 'e') → 'we'

        # Builds the final vocabulary

        # Only done once during training

        # Convert to list of words if string
        if isinstance(corpus, str):
            corpus = corpus.lower().split()
        
        # Count word frequencies
        word_freqs = Counter(corpus)
        # Initialize vocabulary with character-level tokens
        # Add </w> to mark end of word
        vocab = {}
        for word, freq in word_freqs.items():
            # Split into characters and add end-of-word marker
            tokens = list(word) + ['</w>']
            vocab[' '.join(tokens)] = freq
        
        print(f"Initial vocabulary size: {len(set(token for word in vocab.keys() for token in word.split()))}")
        print(f"Example initial splits: {list(vocab.keys())[:3]}\n")
        # print(vocab)
        # Perform merges
        for i in range(self.num_merges):
            # Count all pairs
            pairs = defaultdict(int)
            for word, freq in vocab.items():
                symbols = word.split()
                for pair in self.get_pairs(symbols):
                    pairs[pair] += freq
            
            if not pairs:
                break
            
            # Find most frequent pair
            best_pair = max(pairs, key=pairs.get)
            print(f"Merge {i+1}: ('{best_pair[0]}', '{best_pair[1]}') → '{best_pair[0]+best_pair[1]}' (frequency: {pairs[best_pair]})")
            
            # Store merge rule
            self.merges[best_pair] = best_pair[0] + best_pair[1]
            
            # Apply merge to vocabulary
            new_vocab = {}
            bigram = ' '.join(best_pair)
            replacement = best_pair[0] + best_pair[1]
            
            for word, freq in vocab.items():
                new_word = word.replace(bigram, replacement)
                new_vocab[new_word] = freq
            
            vocab = new_vocab
        # print (self.merges)
        # Build final vocabulary
        self.vocab = set()
        for word in vocab.keys():
            self.vocab.update(word.split())
        
        print(f"\nFinal vocabulary size: {len(self.vocab)}")
        print(f"Sample tokens: {sorted(list(self.vocab))[:20]}")
    
    def encode(self, word):
        """
        Encode a word using learned BPE merges.
        
        Args:
            word: String to encode
            
        Returns:
            List of tokens
        """
    # Starts with character-level tokens of the input word

    # Applies pre-learned merge rules in the same order they were discovered during training

    # Doesn't learn anything new - just uses existing rules

    # Can be called many times on new data

        word = word.lower()
        # Start with characters
        tokens = list(word) + ['</w>']
        
        # Apply merges in order
        while len(tokens) > 1:
            pairs = self.get_pairs(tokens)
            if not pairs:
                break
            
            # Find first applicable merge rule
            # (We need to apply merges in the order they were learned)
            bigram = None
            for merge in self.merges:
                if merge in pairs:
                    bigram = merge
                    break
            
            if not bigram:
                break
            
            # Apply the merge
            first, second = bigram
            new_tokens = []
            i = 0
            while i < len(tokens):
                if i < len(tokens) - 1 and tokens[i] == first and tokens[i+1] == second:
                    new_tokens.append(first + second)
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
        
        return tokens

print("BPE implementation ready!")

BPE implementation ready!


## 5. Training BPE on a Sample Corpus

Let's train our BPE tokenizer on a small corpus!

In [3]:
# Sample corpus
corpus = """
low low low low low
lower lower
lowest lowest lowest
newer newer newer newer
newest newest newest newest newest newest
wider wider wider
widest widest widest
higher higher
highest
"""

# Initialize and train BPE
bpe = BytePairEncoding(num_merges=15)
bpe.train(corpus)

Initial vocabulary size: 13
Example initial splits: ['l o w </w>', 'l o w e r </w>', 'l o w e s t </w>']

Merge 1: ('w', 'e') → 'we' (frequency: 15)
Merge 2: ('s', 't') → 'st' (frequency: 13)
Merge 3: ('st', '</w>') → 'st</w>' (frequency: 13)
Merge 4: ('r', '</w>') → 'r</w>' (frequency: 11)
Merge 5: ('l', 'o') → 'lo' (frequency: 10)
Merge 6: ('e', 'we') → 'ewe' (frequency: 10)
Merge 7: ('n', 'ewe') → 'newe' (frequency: 10)
Merge 8: ('newe', 'st</w>') → 'newest</w>' (frequency: 6)
Merge 9: ('i', 'd') → 'id' (frequency: 6)
Merge 10: ('w', 'id') → 'wid' (frequency: 6)
Merge 11: ('wid', 'e') → 'wide' (frequency: 6)
Merge 12: ('lo', 'w') → 'low' (frequency: 5)
Merge 13: ('low', '</w>') → 'low</w>' (frequency: 5)
Merge 14: ('newe', 'r</w>') → 'newer</w>' (frequency: 4)
Merge 15: ('lowe', 'st</w>') → 'lowest</w>' (frequency: 3)

Final vocabulary size: 12
Sample tokens: ['e', 'g', 'h', 'i', 'low</w>', 'lowe', 'lowest</w>', 'newer</w>', 'newest</w>', 'r</w>', 'st</w>', 'wide']


### Understanding the Merges

Notice how BPE learned meaningful subword units:
- It merged common endings like "est"
- It learned common prefixes and roots
- Frequent words get merged into single tokens

## 6. Encoding Words with BPE

Now let's use our trained BPE to encode some words!

In [4]:
# Test encoding
test_words = ['lowest', 'newer', 'highest', 'lower', 'newer', 'fastest']

print("BPE Encoding Results:")
print("=" * 50)
for word in test_words:
    tokens = bpe.encode(word)
    print(f"'{word}' → {tokens}")
    print(f"  ({len(word)} chars → {len(tokens)} tokens)\n")

BPE Encoding Results:
'lowest' → ['lo', 'we', 'st</w>']
  (6 chars → 3 tokens)

'newer' → ['newer</w>']
  (5 chars → 1 tokens)

'highest' → ['h', 'i', 'g', 'h', 'e', 'st</w>']
  (7 chars → 6 tokens)

'lower' → ['lo', 'we', 'r</w>']
  (5 chars → 3 tokens)

'newer' → ['newer</w>']
  (5 chars → 1 tokens)

'fastest' → ['f', 'a', 'st', 'e', 'st</w>']
  (7 chars → 5 tokens)



### Observations:

- **Seen words**: Words in the training corpus are encoded efficiently
- **Unseen words**: Even words not in the training corpus (like 'fastest') can be encoded!
- **Shared subwords**: Similar words share common subword tokens (like 'low' in 'lower' and 'lowest')
- **Graceful degradation**: In the worst case, we fall back to character-level encoding

## 7. Visualizing BPE in Action

Let's create a visual comparison of different tokenization methods:

In [5]:
def compare_tokenizations(text, bpe_tokenizer):
    """
    Compare character, word, and BPE tokenization.
    """
    words = text.lower().split()
    
    # Character-level
    char_tokens = list(text.lower())
    
    # Word-level
    word_tokens = words
    
    # BPE
    bpe_tokens = []
    for word in words:
        bpe_tokens.extend(bpe_tokenizer.encode(word))
    
    print("Tokenization Comparison")
    print("=" * 70)
    print(f"Original text: '{text}'\n")
    
    print(f"Character-level ({len(char_tokens)} tokens):")
    print(f"  {char_tokens}\n")
    
    print(f"Word-level ({len(word_tokens)} tokens):")
    print(f"  {word_tokens}\n")
    
    print(f"BPE ({len(bpe_tokens)} tokens):")
    print(f"  {bpe_tokens}\n")
    
    print("Summary:")
    print(f"  Character tokens: {len(char_tokens)}")
    print(f"  Word tokens: {len(word_tokens)}")
    print(f"  BPE tokens: {len(bpe_tokens)} ← Best balance!")

# Test it
test_text = "the newest lowest highest"
compare_tokenizations(test_text, bpe)

Tokenization Comparison
Original text: 'the newest lowest highest'

Character-level (25 tokens):
  ['t', 'h', 'e', ' ', 'n', 'e', 'w', 'e', 's', 't', ' ', 'l', 'o', 'w', 'e', 's', 't', ' ', 'h', 'i', 'g', 'h', 'e', 's', 't']

Word-level (4 tokens):
  ['the', 'newest', 'lowest', 'highest']

BPE (14 tokens):
  ['t', 'h', 'e', '</w>', 'newest</w>', 'lo', 'we', 'st</w>', 'h', 'i', 'g', 'h', 'e', 'st</w>']

Summary:
  Character tokens: 25
  Word tokens: 4
  BPE tokens: 14 ← Best balance!


## 8. Key Advantages of BPE

### 1. **Vocabulary Efficiency**
- Controlled vocabulary size (you choose num_merges)
- Much smaller than word-level
- Much more meaningful than character-level

### 2. **Handles Unknown Words**
- Can tokenize any word, even if not in training data
- Falls back to characters in worst case
- No "unknown token" needed!

### 3. **Captures Morphology**
- Learns prefixes (un-, re-, pre-)
- Learns suffixes (-ing, -ed, -est)
- Learns common roots

### 4. **Data-Driven**
- No linguistic knowledge required
- Adapts to any language
- Learns from actual text distribution

### 5. **Compression**
- Reduces sequence length compared to characters
- Maintains meaning better than characters
- Faster training and inference for models

## 9. BPE in Modern Language Models

BPE is used in many famous models:

- **GPT-2/GPT-3**: Uses BPE with ~50,000 merges
- **RoBERTa**: Uses byte-level BPE
- **BART**: Uses BPE tokenization
- **T5**: Uses SentencePiece (BPE variant)

### Real-world Considerations:

1. **Byte-level BPE**: Instead of characters, start with bytes (0-255)
   - Handles any Unicode character
   - Used in GPT-2

2. **Pre-tokenization**: Often split on whitespace and punctuation first

3. **Special tokens**: Add special tokens like `<|endoftext|>`, `<PAD>`, etc.

4. **Vocabulary size**: Typically 30,000-50,000 tokens
   - Smaller = longer sequences, less memory
   - Larger = shorter sequences, more parameters

## 10. Hands-on Exercise

Try training BPE on your own text!

In [6]:
# Exercise: Train BPE on a custom corpus

# TODO: Replace this with your own text!
# my_corpus = """
# machine learning is amazing
# deep learning is powerful
# learning algorithms are interesting
# neural networks are learning systems
# unsupervised learning learns patterns
# """
my_corpus = """
Do you ever feel like a plastic bag
Drifting through the wind, wanting to start again?
Do you ever feel, feel so paper thin
Like a house of cards, one blow from cavin' in?
Do you ever feel already buried deep?
Six feet under screams, but no one seems to hear a thing
Do you know that there's still a chance for you?
'Cause there's a spark in you
You just gotta ignite the light
And let it shine
Just own the night
Like the Fourth of July
'Cause baby, you're a firework
Come on, show 'em what you're worth
Make 'em go, "Oh, oh, oh"
As you shoot across the sky
Baby, you're a firework
Come on, let your colors burst
Make 'em go, "Oh, oh, oh"
You're gonna leave 'em all in awe, awe, awe
You don't have to feel like a waste of space
You're original, cannot be replaced
If you only knew what the future holds
After a hurricane comes a rainbow
Maybe a reason why all the doors are closed
So you could open one that leads you to the perfect road
Like a lightning bolt, your heart will glow
And when it's time, you'll know
You just gotta ignite the light
And let it shine
Just own the night
Like the Fourth of July
'Cause baby, you're a firework
Come on, show 'em what you're worth
Make 'em go, "Oh, oh, oh"
As you shoot across the sky
Baby, you're a firework
Come on, let your colors burst
Make 'em go, "Oh, oh, oh"
You're gonna leave 'em all in awe, awe, awe
Boom, boom, boom
Even brighter than the moon, moon, moon
It's always been inside of you, you, you
And now it's time to let it through
'Cause baby, you're a firework
Come on, show 'em what you're worth
Make 'em go, "Oh, oh, oh"
As you shoot across the sky
Baby, you're a firework
Come on, let your colors burst
Make 'em go, "Oh, oh, oh"
You're gonna leave 'em all in awe, awe, awe
Boom, boom, boom
Even brighter than the moon, moon, moon
Boom, boom, boom
Even brighter than the moon, moon, moon
"""
# Train a BPE tokenizer
my_bpe = BytePairEncoding(num_merges=20)
my_bpe.train(my_corpus)

# Test it
print("\nTest encoding:")
test_words = ['you', 'machine', 'neural', 'supervised']
for word in test_words:
    print(f"'{word}' → {my_bpe.encode(word)}")

Initial vocabulary size: 29
Example initial splits: ['d o </w>', 'y o u </w>', 'e v e r </w>']

Merge 1: ('e', '</w>') → 'e</w>' (frequency: 76)
Merge 2: (',', '</w>') → ',</w>' (frequency: 57)
Merge 3: ('o', 'u') → 'ou' (frequency: 42)
Merge 4: ('y', 'ou') → 'you' (frequency: 36)
Merge 5: ('t', '</w>') → 't</w>' (frequency: 36)
Merge 6: ('t', 'h') → 'th' (frequency: 32)
Merge 7: ('o', 'n') → 'on' (frequency: 24)
Merge 8: ('a', '</w>') → 'a</w>' (frequency: 21)
Merge 9: ('s', '</w>') → 's</w>' (frequency: 20)
Merge 10: ('i', 'n') → 'in' (frequency: 18)
Merge 11: ('o', 'h') → 'oh' (frequency: 18)
Merge 12: ('o', 'm') → 'om' (frequency: 17)
Merge 13: ('th', 'e</w>') → 'the</w>' (frequency: 16)
Merge 14: ('you', '</w>') → 'you</w>' (frequency: 15)
Merge 15: ('r', '</w>') → 'r</w>' (frequency: 15)
Merge 16: ('r', 'e</w>') → 're</w>' (frequency: 15)
Merge 17: ('you', ''') → 'you'' (frequency: 14)
Merge 18: ('o', 'r') → 'or' (frequency: 14)
Merge 19: ('e', 'm') → 'em' (frequency: 13)
Merge 2

### Exercise Questions:

1. What subword units did BPE learn from your corpus?
2. Try encoding a word that wasn't in your training corpus. How does BPE handle it?
3. Experiment with different values of `num_merges`. How does it affect the results?
4. Add more examples of related words (e.g., "learn", "learned", "learner"). Does BPE capture the relationship?

## 11. Advanced Topics

### BPE Variants:

1. **Byte-Level BPE**:
   - Operates on bytes instead of characters
   - Vocabulary of 256 base tokens (all possible bytes)
   - Can handle any Unicode text

2. **WordPiece**:
   - Used in BERT
   - Instead of frequency, uses likelihood to choose merges
   - Adds special prefix (##) for continuing subwords

3. **SentencePiece**:
   - Treats text as Unicode characters
   - No pre-tokenization needed
   - Language-agnostic

### Implementation Tips:

- **Caching**: Store merge results to speed up encoding
- **Parallel processing**: Train on multiple documents simultaneously
- **Incremental training**: Update vocabulary with new data
- **Vocabulary pruning**: Remove rare tokens to save memory

## 12. Using Production BPE Libraries

For real projects, use optimized libraries like `tokenizers` from Hugging Face:

In [7]:
conda install tokenizers

Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [8]:
# Note: You'll need to install the tokenizers library first
# pip install tokenizers

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize a tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

# Configure the trainer
trainer = BpeTrainer(special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"], vocab_size=1000)

# Train on files
# tokenizer.train(files=["corpus.txt"], trainer=trainer)

# Or train on an iterator
corpus_list = [
    "the quick brown fox jumps over the lazy dog",
    "machine learning is fascinating",
    "natural language processing with transformers"
]
tokenizer.train_from_iterator(corpus_list, trainer=trainer)

# Encode text
output = tokenizer.encode("machine learning is fascinating")
print(f"Tokens: {output.tokens}")
print(f"IDs: {output.ids}")




Tokens: ['machine', 'learning', 'is', 'fascinating']
IDs: [93, 92, 55, 86]


## 13. Summary and Key Takeaways

### What We Learned:

1. **Tokenization** converts text into processable units
2. **BPE** finds the sweet spot between character and word-level tokenization
3. The algorithm is **simple**: iteratively merge the most frequent pairs
4. BPE is **data-driven** and learns meaningful subword units
5. It **handles unknown words** gracefully
6. BPE is used in **most modern language models**

### When to Use BPE:

✅ Building language models  
✅ Working with morphologically rich languages  
✅ Need to handle unknown words  
✅ Want to control vocabulary size  
✅ Working with limited training data  

### Further Reading:

- Original BPE paper: "Neural Machine Translation of Rare Words with Subword Units" (Sennrich et al., 2016)
- GPT-2 paper for byte-level BPE: "Language Models are Unsupervised Multitask Learners"
- Hugging Face tokenizers documentation: https://huggingface.co/docs/tokenizers/

---

## Congratulations! 🎉

You now understand how BPE tokenization works and how it powers modern language models. Try experimenting with different corpora and parameters to build intuition!